In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
from pandas.tseries.offsets import BDay

In [2]:
# Get implied volatility for one month (30 days or 21 business days) ahead
# go through remained option expiry date and get atm iv for each of the expiry date
# go through the target dates. if there is option with the same expiry date use its atm implied vol if not interpolate to get its implied vol

def quadratic(x, a, b, c):
    return a * x**2 + b * x + c

DATE_FORMATE = '%Y-%m-%d'
PERIOD = 21
gcft = yf.Ticker('GLD')
expiry_dates = gcft.options
expiry_dates = [datetime.strptime(ed, DATE_FORMATE).date() for ed in expiry_dates]

latest_asset_rec = gcft.history(period='1d').reset_index()
query_date = pd.Timestamp(latest_asset_rec['Date'].values[0]).date()
asset_price = latest_asset_rec['Close'].iloc[0]
target_date = (query_date + BDay(PERIOD)).date()

# print(f'before: {expiry_dates}')
if target_date in expiry_dates:
    expiry_dates = [ed for ed in expiry_dates if ed >= target_date]
else:
    before = [ed for ed in expiry_dates if ed < target_date]
    expiry_dates = expiry_dates[len(before)-1:]
# print(f'after: {expiry_dates}')

# get ATM iv for each of the expiry_dates
atm_iv = {}
for ed in expiry_dates:
    # print(ed)
    ed = ed.strftime(DATE_FORMATE)
    options = gcft.option_chain(ed)
    calls = options.calls
    puts = options.puts

    # get ATM IV
    strikes = np.concatenate([calls['strike'], puts['strike']])
    ivs = np.concatenate([calls['impliedVolatility'].fillna(0), puts['impliedVolatility'].fillna(0)])

    # Sort by strike price
    sorted_indices = np.argsort(strikes)
    strikes = strikes[sorted_indices]
    ivs = ivs[sorted_indices]

    params, _ = curve_fit(quadratic, strikes, ivs)
    smooth_strikes = np.linspace(min(strikes), max(strikes), 100)
    smooth_iv = quadratic(smooth_strikes, *params)

    # plt.scatter(calls['strike'], calls['impliedVolatility'], color='red', label='calls IV', alpha=0.6)
    # plt.scatter(puts['strike'], puts['impliedVolatility'], color='blue', label='puts IV', alpha=0.6)

    # plt.plot(smooth_strikes, smooth_iv, color="black", label="Smoothed IV Curve", linewidth=2)
    # plt.xlabel("Strike Price")
    # plt.ylabel("Implied Volatility")

    atm_iv.update({ed:quadratic(asset_price, *params)})

# interpolate with nearest expiry dates to target date
target_iv_list = []
while target_date <= expiry_dates[-1]:
    td = target_date.strftime(DATE_FORMATE)
    if target_date in expiry_dates:
        target_iv_list.append({'target_date': td, 
                               'impliedVol': atm_iv[td]})
    else:
        before = [ed for ed in expiry_dates if ed < target_date][-1]
        after = [ed for ed in expiry_dates if ed > target_date][0]
        
        target_iv_list.append({'target_date': td, 
                               'impliedVol': (target_date - before).days/(after - before).days * (atm_iv[after.strftime(DATE_FORMATE)] - atm_iv[before.strftime(DATE_FORMATE)]) + atm_iv[before.strftime(DATE_FORMATE)] })
    
    target_date = (target_date + BDay(1)).date()
target_iv_df = pd.DataFrame(target_iv_list)    
target_iv_df    

,target_date,impliedVol
0,2025-05-09,0.243131
1,2025-05-12,0.254418
2,2025-05-13,0.258180
3,2025-05-14,0.261943
4,2025-05-15,0.265705
...,...,...
436,2027-01-11,0.209574
437,2027-01-12,0.209487
438,2027-01-13,0.209400
439,2027-01-14,0.209312
